<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://erachelson.github.io/RLclass_MVA/">https://erachelson.github.io/RLclass_MVA/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Chapter 5: From Monte Carlo policy gradients to actor-critic algorithms</div>

So far, we have been mostly concerned with the problem of function approximation, that is "how do we store $q(s,a)$ in a convenient way, that is amenable to learning, retrieval and optimization?". A specific feature of the problems we tackled was that they had few discrete actions, making the dependency on $s$ the key difficulty. In particular, when looking for a $q$-greedy action, the $\max_{a\in A}$ problem had a straightforward solution as we could iterate through actions and retain the best one. We now turn to a more general case where the actions are too numerous to be enumerated (either because the action space is continuous or because it just has too many actions). Too many states motivated the introduction of function approximators for $v$ and $q$; too many actions similarly lead to function approximation for $\pi$. In the present chapter, we consider the general policy space (with possible approximation) and ask the question "how do we find a monotonically improving sequence of policies?".

This chapter and the next are tightly linked. The only reason they don't form a single chapter is to let the reader breathe, but they should probably be merged. In this chapter, we turn back to a criterion for optimality we introduced in the first chapters, namely the average value of a policy across (starting) states. We explore the question of writing the gradient of this criterion with respect to policy parameters, which leads us to a first formulation of this policy gradient a family of associated algorithms.

The next chapter will pick up from where we stopped and will keep on elaborating on the same gradient estimators.

<div class="alert alert-success">

**Learning outcomes**   
By the end of this chapter, you should be able to:
- recall and explain the expression of the Monte Carlo policy gradient for infinite horizon problems,
- explain the problem of gradient estimate variance and the introduction of baselines,
- implement a REINFORCE algorithm for discrete action spaces,
- explain what an actor-critic algorithm is,
- implement an A2C algorithm for discrete action spaces.

Additionally, after doing the homework, you should be able to:
- use vectorized environments to accelerate sample collection in policy gradient algorithms,
- implement REINFORCE and A2C for continuous action spaces,
- derive the policy gradient theorem for finite horizon problems,
- explain and implement a generalized advantage estimation method within a policy gradient algorithm,
- derive the formulation of off-policy policy gradients,
- implement natural policy gradients, TRPO and PPO,
- explain and implement the "OpenAI evolution strategy" and the "Canonical evolution strategy" for gradient-free direct policy search.
</div>

# Policy gradient methods

<div class="alert alert-success">

**Bottomline question:**   
The previous chapters have focussed on *action-value methods*; they aimed at estimating $q^*$ in order to deduce $\pi^*$, or they jointly optimized $q$ and $\pi$. Could we directly optimize $\pi$?
</div>

Suppose we have a policy $\pi_\theta$ parameterized by a vector $\theta$. Our goal is to find the parameter $\theta^*$ corresponding to $\pi^*$.

<div class="alert alert-warning">
    
**Exercise:**  
Recall the FrozenLake environment.  
How many states and how many actions were there in this environment? 
What would be a policy parameterization which does not make any approximation (ie. that can represent any policy in the policy space) for stationary, memoryless, stochastic policies?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

There are 16 states and 4 actions in the FrozenLake game.

A stationary, memoryless, stochastic policy is a mapping from $S$ to $\Delta_A$. Since $S$ and $A$ are discrete, it can be represented in tabular form as:
$$\pi = \left[ \begin{array}{cccc}
\pi(a_0|s_0) & \pi(a_1|s_0) & \pi(a_2|s_0) & 1 - \sum_{i=0}^2 \pi(a_i|s_0) \\
\ldots & & & \\
\pi(a_0|s_{|S|}) & \pi(a_1|s_{|S|}) & \pi(a_2|s_{|S|}) & 1 - \sum_{i=0}^2 \pi(a_i|s_{|S|})
\end{array} \right].$$

This parameterization enables representing any stochastic policy. It involves $|A|-1=3$ parameters per line, and $|S|=16$ lines, so in total $3\times 16 = 58$ parameters.

As previously, parameterization does not necessarily involve approximation! As the action set will become large or continuous, parameterization will enable generalization across actions at the cost of approximation, but a tabular representation is also a policy parameterization.
</details>

Remarks:
- $\pi_\theta$ might not be able to represent $\pi^*$. We will take a shortcut and call $\pi^*$ the best policy among the $\pi_\theta$ ones.
- For discrete state and action spaces, the tabular policy representation is a special case of policy parameterization.
- Policy parameterization is a (possibly useful) way of introducing prior knowledge on the set of the desired policies.
- The optimal deterministic policies might not belong to the policy subspace of $\pi_\theta$, thus it makes sense to consider stochastic policies for $\pi_\theta$.
- It makes even more sense to consider stochastic policies that it opens the family of environments that we can tackle, like partially observable MDPs or multi-player games.

For stochastic policies, we shall write $\pi_\theta(a|s)$.

In the remainder of the chapter, we will assume that $\pi_\theta(a|s)$ is differentiable with respect to $\theta$.

Suppose now we define some performance metric $J(\pi_\theta) = J(\theta)$. If $J$ is differentiable and a stochastic estimate $\tilde{\nabla}_\theta J(\theta)$ of the gradient is available, then we can define the stochastic gradient ascent update procedure:
$$\theta \leftarrow \theta + \alpha \tilde{\nabla}_\theta J(\theta).$$

We will call **policy gradient methods** all methods that follow such a procedure (whether or not they also learn a value function or not).

<div class="alert alert-success">

**Policy gradient method**   
We call **policy gradient method** any method that performs stochastic gradient ascent on the policy's parameters.  
Given a stochastic estimate $\tilde{\nabla}_\theta J(\theta)$ of a policy's performance criterion with respect to the policy's parameters, such a method implements the update procedure: 
$$\theta \leftarrow \theta + \alpha \tilde{\nabla}_\theta J(\theta).$$
</div>

Remarks: 
- Note that $J$ is a generic criterion. For example, $J$ could be defined as the $\gamma$-discounted value of a starting state (or a distribution of starting states), or as the undiscounted reward over a certain horizon, or as the average reward.
- Note that this family of methods can use any gradient estimate for $\tilde{\nabla}_\theta J(\theta)$: formal calculus, finite differences, automated differentiation, evolution strategies, etc.
- Why is it interesting to look at methods which explicitly store a policy function? Because the evaluation of the policy in a given state $s$ does not require the maximization step ($\max_a Q(s,a)$), which might be computationally costly, especially for continuous actions. Instead, it replaces it with a call to $\pi_\theta(s)$ (or a draw from $\pi_\theta(a|s)$). This argument makes actor-critic architectures or direct policy search a method of choice for continuous actions domains (especially common in Robotics) and Policy Gradient is one of them.
- When do policy gradient approaches outperform value-based ones? It's hard to give a precise criterion; it really depends on the problem. One thing that comes into play is how easy it is to approximate the optimal policy or the optimal value function. If one is simpler than the other (by "simpler", we mean "it is easier to find a parameterization whose spanned function space almost includes the function to approximate"), then it is a good heuristic to try to approximate it. But this criterion might itself be hard to assess.

<div class="alert alert-warning">

**Exercise**  
From the class on Markov Decision Processes, can you recall a scalar criterion $J(\pi)$ whose optimization is provably equivalent to finding a policy that dominates any other one in every state?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Provided $\rho_0$ has non-zero probability mass on all states, an optimal policy is a solution to $\max_\pi J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[v^\pi(s_0)]$.

This makes optimizing $J(\pi)$ a legitimate goal for finding optimal policies: from now on we will work with $J(\pi)$.
</details>

This section served as a general introduction to two families of estimators for policy gradients, which we will study separately.   

First, in the present chapter, we will look at **Monte Carlo policy gradient** methods. These methods rely on simulating one or several full trajectories with $\pi$ to compute $\nabla J$. They are quite intuitive and make for a good first contact with the problem of estimating $\nabla J$. These methods include the very well-known REINFORCE method and its modern offsprings, but also most of the literature on evolutionary reinforcement learning.  

Then, in the next chapter, we will follow the same kind of reasoning the enabled moving from $v(s) = \sum_t \gamma^t r_t$ to the Bellman equation, and will study how $\nabla J$ can be expressed as a quantity one can estimate from independent $(s,a,r,s')$ samples. All the corresponding algorithms will be motivated by the famous **policy gradient theorem** which will enable an interesting connection with what we have already covered in approximate dynamic programming.

These two families are split into two chapters to keep each chapter's length reasonable. But it is important to stress this out from the start: **both aim to estimate the same quantity $\nabla J$**!

# A Monte Carlo policy gradient estimator

Recall the policy optimization objective defined in previous chapters, given a distribution $p_0$ on states:  
$$J(\pi) = \mathbb{E}_{s \sim p_0} \left[ v^{\pi} (s) \right].$$
Or equivalently:  
$$J(\pi) = \mathbb{E}_{(s_i,a_i)_{i \in \mathbb{N}}} \left[ \sum_{t=0}^\infty \gamma^t r(s_t,a_t)  | \pi, p_0 \right].$$

We will write $\tau = (s_t,a_t)_{t\in \mathbb{N}}$ and thus $J(\pi) = \mathbb{E}_{\tau} \left[ \sum_{t=0}^\infty \gamma^t r(s_t,a_t)  | \pi, p_0 \right]$.

One could imagine a *finite differences* approach to estimate $\nabla_\theta J(\theta)$, but this would require trying out a series of increments $\Delta \theta$ which quickly becomes impractical (because the increment size is hard to tune, especially in stochastic systems, and also because of the sample inefficiency of the approach).

Remark:
- Let's not discard finite difference methods too quickly. They have their merits and showed great successes through methods such as [PEGASUS (Ng and Jordan, 2000)](https://arxiv.org/abs/1301.3878). Also, having random $\Delta \theta$ drawn from a Gaussian distribution is essentially what Evolution Strategies do, and [Salimans et al. (2017)](https://arxiv.org/abs/1703.03864) or [Chrabaszcz et al. (2018)](https://arxiv.org/abs/1802.08842) illustrated how that could be a scalable method to obtain gradient estimates in RL. We won't cover these topics here and leave them as exercises.  

The key result of this chapter is that one can express the gradient of $J(\theta)$ as directly proportional to the value of a trajectory and the gradient of $\pi$:
<div class="alert alert-success">

**Monte Carlo policy gradient**  
Given trajectories $\tau=(s_t,a_t)_{t\in [0,\infty]}$ drawn according to policy $\pi$, 
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_{t=0}^\infty \gamma^t G_t \nabla_\theta \log \pi_\theta(a_t|s_t) \right],$$
where $G_t = G^\pi(s_t,a_t) = \sum_{t'\geq t} \gamma^{t'-t} r_{t'}$ is the return random variable from the state-action pair visited at time step $t$.
</div>

The interpretation of this estimator is very straightforward:  
To increase the average value of policy $\pi_\theta$ over the starting distribution $p_0$, we should change $\theta$ in a direction that is a linear combination of the $\nabla_\theta \log \pi(a|s)$, where the coefficients are the expected outcomes $G^\pi(s,a)$ of picking action $a$ in $s$.  
Since $\nabla_\theta \log \pi(a|s)$ is a direction that increases the log probability of $a$ in $s$, we can rephrase the last sentence. The policy gradient tells us:  
**To increase the value of the current policy, we should increase the log-probability of $a$ in $s$ in proportion to the expected outcome of $a$ in $s$.**

Let us derive this result quite naturally.

Let us consider trajectories $\tau = (s_0,a_0,r_0,...)$ drawn according to $\pi$ from the starting state. Each of these trajectories has an overall payoff of $G(\tau) = \sum_t \gamma^t r_t$, and is drawn with probability density $p(\tau|\theta)$. Then the objective function can be written:
\begin{align}
J(\theta) &= \mathbb{E}_\tau \left[ G(\tau) | \theta \right]\\
 &= \int G(\tau) p(\tau | \theta) d\tau
\end{align}

So the objective function's gradient is:
\begin{align}
\nabla_\theta J(\theta) &= \nabla_\theta \int G(\tau) p(\tau|\theta) d\tau,\\
 &= \int G(\tau) \nabla_\theta p(\tau|\theta) d\tau,\\
 &= \int G(\tau) p(\tau|\theta) \frac{\nabla_\theta p(\tau|\theta)}{p(\tau|\theta)} d\tau,\\
 &= \mathbb{E}_\tau \left[ G(\tau) \nabla_\theta \log p(\tau|\theta) \right].
\end{align}

We have used the fact that $\nabla_\theta p(\tau|\theta) = p(\tau|\theta) \nabla_\theta \log p(\tau|\theta)$, sometimes known as the *nabla-log* trick.  
Let us study the $\nabla_\theta \log p(\tau|\theta)$, term along a series of remarks.

**Remark 1: law of $s_{t+1},a_{t+1}$ given the policy and history.**  
One has $p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) p(a_{t+1} | s_{t+1}, (s_i,a_i)_{i \in [0,t]}, \theta)$.  
But the transition model is Markovian, so $p(s_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | s_t, a_t)$.  
And the law of $a_{t+1}$ is given by the policy, so $p(a_{t+1} | s_{t+1}, (s_i,a_i)_{i \in [0,t]}, \theta) = \pi_\theta(a_{t+1}|s_{t+1})$.  
Consequently:
$$p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta) = p(s_{t+1} | s_t, a_t) \pi_\theta(a_{t+1}|s_{t+1}).$$

**Remark 2: probability density of a trajectory.**  
Recall that $p(\tau|\theta) = p((s_t,a_t)_{t\in [0,\infty]}|\theta)$.  
This joint probability can be decomposed into conditional probabilities: $$p(\tau|\theta) = p(s_0,a_0|\theta) \prod_{t=0}^\infty p(s_{t+1},a_{t+1} | (s_i,a_i)_{i \in [0,t]}, \theta).$$ 
The previous remarks allows us to simplify to: $$p(\tau|\theta) = p(s_0,a_0|\theta) \prod_{t=0}^\infty p(s_{t+1} | s_t, a_t) \pi_\theta(a_{t+1}|s_{t+1}).$$ 
By expanding the first term into $p(s_0)\pi_\theta(a_0|s_0)$ and reordering the terms inside the product, we obtain:
$$p(\tau|\theta) = p(s_0) \prod_{t=0}^\infty p(s_{t+1} | s_t, a_t) \pi_\theta(a_t|s_t).$$

**Remark 3: grad-log-prob of a trajectory.**  
Now let us consider the full $\nabla_\theta \log p(\tau|\theta)$ term. The previous remarks tell us that  
$$\nabla_\theta \log p(\tau|\theta) = \nabla_\theta \log p(s_0) + \sum_{t=0}^\infty \left[ \nabla_\theta \log p(s_{t+1} | s_t, a_t) + \nabla_\theta \log \pi_\theta(a_t|s_t)\right].$$
But the initial state distribution and the transition model do not depend on $\theta$, so this expression boils down to:
$$\nabla_\theta \log p(\tau|\theta) = \sum_{t=0}^\infty \nabla_\theta \log \pi_\theta(a_t|s_t).$$

So eventually, we have a first version of a policy gradient estimator:
<div class="alert alert-success">

**Monte Carlo policy gradient (first naive version)**  
Given trajectories $\tau=(s_t,a_t)_{t\in [0,\infty]}$ drawn according to policy $\pi$, and evaluated through a criterion $G(\tau)$, 
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ G(\tau) \sum_{t=0}^\infty \nabla_\theta \log \pi_\theta(a_t|s_t) \right].$$
</div>

This estimator provides a straightforward algorithm. We could play policy $\pi_\theta$ from $p_0$, and collect pairs of states and actions $(s,a)$ along trajectories $\tau$. After each trajectory terminates, we sum the rewards obtained to get $G(\tau)$, then iterate over all encountered states and actions to sum the $G(\tau) \nabla_\theta \log \pi_\theta(a|s)$ terms. This yields an ascent direction: we can take a stochastic gradient ascent step, and repeat.

This estimator is called **Monte Carlo** because it relies on a (Monte Carlo) simulation of $\pi$ to obtain realizations of $G(\tau)$, and uses them as weights of $\nabla \log \pi(a|s)$ in each encountered state.

**Interpretation**  
Let us consider a fixed $s$.  
Then $\nabla_\theta \log \pi_\theta(a|s)$ is the answer to "in what direction should I change $\theta$ to increase the log probability of taking action $a$ in $s$?".  
The expression above tells us that in order to improve the value of $\pi_\theta$, we should change $\theta$ in a direction that is a linear combination of all $\nabla_\theta \pi_\theta(a|s)$, giving more weight to action $a$ in state $s$ in proportion to the value $G(\tau)$ of the trajectory they belong too.   
In even simpler words: if a trajectory was better than another, its action probabilities should be reinforced (that's not the reason for the name of the algorithm, but it could have been).

But we can do a bit better than this estimator.

**Remark 4: the expected grad-log-prob lemma.**  
Let us take a step aside and consider the expectation of $\nabla_\theta \log \pi_\theta(a|s)$.  
In a given $s$:
\begin{align}
\mathbb{E}_{a\sim \pi_\theta(s)} [ \nabla_\theta \log \pi_\theta(a|s) ] &= \int_A \pi_\theta(a|s) \nabla_\theta \log \pi_\theta(a|s) da\\
 &= \int_A \nabla_\theta \pi_\theta(a|s) da\\
 &= \nabla_\theta \int_A \pi_\theta(a|s) da\\
 &= \nabla_\theta 1\\
 &= 0
\end{align}
So, for any distribution $\rho$ on states, we get that: 
$$\mathbb{E}_{\substack{s\sim\rho \\ a\sim \pi}} \left[ \nabla_\theta \log \pi_\theta(a|s) \right] = \mathbb{E}_{s\sim\rho} \left[ \mathbb{E}_{a\sim \pi} \left[ \nabla_\theta \log \pi_\theta(a|s) \right] \right] = \mathbb{E}_{s\sim\rho} [0]=  0.$$
Actually, we can generalize this immediately: we can multiply $\nabla_\theta \log \pi_\theta(a|s)$ by anything that does not depend on $a$ and obtain the same result:
$$\mathbb{E}_{\substack{s\sim\rho \\ a\sim \pi}} \left[ b(s)\left[ \nabla_\theta \log \pi_\theta(a|s) \right] \right] = 0.$$
This will come in handy in a future section about baselines in policy gradients and will serve as a basis for the exercise on [Generalized Advantage Estimation (Schulman et al., 2016)](https://arxiv.org/abs/1506.02438), but for now it will help us finish improving our policy gradient estimator.

**Back to the policy gradient estimator.**  
Let us replace $G(\tau)$ by its expression in terms of $s_t$ and $a_t$:
$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \left(\sum_{t=0}^\infty \gamma^t r(s_t,a_t) \right) \left( \sum_{t=0}^\infty \nabla_\theta \log \pi_\theta(a_t|s_t) \right) \right] .$$

$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=0}^\infty \gamma^{t'} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

Let us look at one of the terms under the $\sum_{t=0}^\infty$ sum: $\left( \sum_{t'=0}^\infty \gamma^{t'} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t)$.  
It seems awkward that $\nabla_\theta \log \pi_\theta(a_t|s_t)$ be weighted by rewards that were obtained *before* $t$.  
So let us cut the $\sum_{t'=0}^\infty$ in two; the rewards obtained up to $t-1$ and those obtained after $t$:

$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'}) + \sum_{t'=t}^\infty \gamma^{t'} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

Let us develop this sum:
$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right) \nabla_\theta \log \pi_\theta(a_t|s_t) + \sum_{t=0}^\infty \left(\sum_{t'=t}^\infty \gamma^{t'} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

When we look at the first term in the sum, we remark that $\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})$ does not depend on $s_t,a_t$. These are rewards that were obtained in the past, before $t$. By application of the expected grad-log-prob lemma, we have immediately:
$$\mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] = 0$$

<details class="alert ">
    <summary markdown="span"><b>If you are not convinced and want to see the calculation in more detail, click this box to expand it.</b></summary>

\begin{align*}
\mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] &= \sum_{t=0}^\infty \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right]\\
 &= \sum_{t=0}^\infty \mathbb{E}_{(s_i,a_i)_{i\in [0,t]}} \left[ \left(\sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right]\\
  &= \sum_{t=0}^\infty \mathbb{E}_{s_t,a_t | (s_i,a_i)_{i\in [0,t-1]}} \left[ \mathbb{E}_{(s_i,a_i)_{i\in [0,t-1]}} \left[ \sum_{t'=0}^{t-1} \gamma^{t'} r(s_{t'},a_{t'})\right] \nabla_\theta \log \pi_\theta(a_t|s_t) \right]\\
  &= 0
\end{align*}
</details>

So,
$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \left(\sum_{t'=t}^\infty \gamma^{t'} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

And, factoring out the $\gamma^t$,
$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \gamma^t \left(\sum_{t'=t}^\infty \gamma^{t'-t} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

At this stage, it is important to recognize the random variable $G^\pi(s)$ in this expression. Indeed, the sum $\sum_{t'=t}^\infty \gamma^{t'-t} r(s_{t'},a_{t'})$ is exactly $G^\pi(s_t)$. Similarly, one can recognize $G^\pi(s,a)$, keeping in mind that $a$ is drawn according to $\pi(s)$ anyway.

This yields a finer expression for the policy gradient.

<div class="alert alert-success">

**Monte Carlo policy gradient**  
Given trajectories $\tau=(s_t,a_t)_{t\in [0,\infty]}$ drawn according to policy $\pi$, 
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_{t=0}^\infty \gamma^t G_t \nabla_\theta \log \pi_\theta(a_t|s_t) \right],$$
where $G_t = G^\pi(s_t,a_t) = \sum_{t'\geq t} \gamma^{t'-t} r_{t'}$ is the return random variable from the state-action pair visited at time step $t$
</div>

This estimator is called **Monte Carlo** because it relies on a (Monte Carlo) simulation of $\pi$ to obtain realizations of $G^\pi(s,\pi(s))$ in the states visited by $\pi$, and then uses these realizations as weights of $\nabla \log \pi(a|s)$ in each encountered $(s,a)$ pair.

**Interpretation**  
Let us rephrase and refine the previous interpretation.  
Consider a fixed $s$. Then (again) $\nabla_\theta \log \pi(a|s)$ is the answer to "in what direction should I change $\theta$ to increase the log probability of taking action $a$ in $s$?".   
The expression above tells us that in order to improve the value of $\pi_\theta$, we should change $\theta$ in a direction that is a linear combination of all $\nabla_\theta \log\pi(a|s)$, giving more weight to actions that provided large realizations of $G^\pi(s,\pi(s))$. 

We can push the interpretation of $\nabla_\theta \log \pi(a|s) = \frac{\nabla_\theta \pi(a|s)}{\pi(a|s)}$ a bit further. $\nabla_\theta \pi(a_t|s_t)$ is a vector in parameter space that points in the direction of greatest increase of $\pi(a|s)$. The update will encourage taking a step in this direction if the action provided high return (through $G^\pi(s,\pi(s))$), but moving in this direction will be moderated if the action is already picked frequently (through $\pi(a|s)$) so that other actions have a chance also.

# REINFORCE

To compute the gradient estimator introduced previously, we can run the policy within the environment, and this will provide us with trajectories of states and actions following the distribution of $\tau$ under $\pi$. The full trajectory of states-actions-rewards provides a Monte Carlo estimate $G_t$ of $G^\pi(s_t,a_t)$ from any state-action pair $(s_t,a_t)$ traversed by the trajectory. In turn, this allows to estimate $G_t \nabla_\theta \log \pi(a_t|s_t)$ for any of these pair. The sum over all states provides the gradient estimate.

This algorithm, introduced by [Williams (1992)](https://link.springer.com/article/10.1007/BF00992696) is called REINFORCE. It requires a finite-length trajectory and its pseudo-code goes as follows.
1. Initialize policy parameter $\theta$
2. Generate a trajectory by playing $\pi$: $s_0,a_0,r_0,...s_{T}$
3. For $t\in [1, 2, … , T]$:
    1. Estimate return $G_t$
    2. Update policy parameter: $\theta \leftarrow \theta + \alpha \gamma^t G_t \nabla_\theta \log \pi(a_t|s_t)$

Historial note: REINFORCE is not just about "REINFORCE-ing good trajectories", it is an acronym that actually stands for "REward Increment = Nonnegative Factor x Offset Reinforcement x Characteristic Eligibility". Check the paper out if you want to understand what this means.

Modern deep learning libraries don't take gradients as inputs. Instead, they use automated differentiation to compute them for a given expression, given a batch of samples. So we would like to build an objective function such that its gradient estimated on the minibatch composed of the last trajectory's samples $(s,a)$ corresponds to the expression above.

So we want to define $\ell(\theta)$ such that $-\nabla_\theta \mathbb{E}_{s,a} \left[ \ell(\theta,s,a) \right]$ coincides with $\nabla_\theta J(\theta)$. Note that, computationally, the $\gamma^t G_t$ terms are just coefficients in the gradient estimate. So, very simply, taking $\ell(\theta) = q(s,a) \cdot  \log \pi(a|s)$ achieves this goal as long as $q(s_t,a_t)$ matches the value of $\gamma^t G_t$. 

**Super important remark: the loss is not a performance metric.**  
In supervised learning, it is a common practice to monitor the empirical risk along training. 
But $L(\theta) = -\mathbb{E}_{\substack{s\sim\rho \\ a\sim \pi}} \left[ \ell(\theta,s,a) \right]$ is not comparable to the losses one usually encounters in supervised learning in any way!  
In particular, $-L(\theta) \neq J(\theta)$. So it makes little sense monitoring the value of this artificial "loss" function along training, and even less interpreting its value as some performance metric. It is *just* an expression whose gradient coincides with the (opposite of the) policy gradient.

**On sample efficiency.**  
One key feature of policy gradient algorithms is that they are on-policy: they require the data to have been collected by the current policy, and discard this data once a gradient step is taken. Although this might seem sample inefficient, it can turn out to be an acceptable compromise if the policy gradient steps take the policy towards good returns quickly enough.

Let's implement REINFORCE for discrete action spaces.
<div class="alert alert-warning">
    
**Exercise:**  
Declare a neural network approximator for a categorical distribution for $\pi(a|s)$.  
The `forward` method should output action probabilities.  
Define a `sample_action` method that draws from the actions probabilities in state $s$.  
Define a `log_prob` method that returns the log probability of an $(s,a)$ pair.
</div>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class policyNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        state_dim = env.observation_space.shape[0]
        n_action = env.action_space.n
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, n_action)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(dim=0)
        x = F.relu(self.fc1(x))
        action_scores = self.fc2(x)
        return F.softmax(action_scores,dim=1)

    def sample_action(self, x):
        probabilities = self.forward(x)
        action_distribution = Categorical(probabilities)
        return action_distribution.sample().item()

    def log_prob(self, x, a):
        probabilities = self.forward(x)
        action_distribution = Categorical(probabilities)
        return action_distribution.log_prob(a)

<div class="alert alert-warning">
    
**Exercise:**  
Define a class that implements REINFORCE. Instead of drawing a single trajectory as in the pseudo-code above, include an option to draw several trajectories with the same policy. This will better the distribution of $\tau$ and provide less noisy gradients.
</div>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange

class reinforce_agent:
    def __init__(self, config, policy_network):
        self.device = "cuda" if next(policy_network.parameters()).is_cuda else "cpu"
        self.scalar_dtype = next(policy_network.parameters()).dtype
        self.policy = policy_network
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.99
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = torch.optim.Adam(list(self.policy.parameters()),lr=lr)
        self.nb_episodes_per_gradient_step = config['nb_episodes_per_gradient_step'] if 'nb_episodes_per_gradient_step' in config.keys() else 1
    
    def one_gradient_step(self, env):
        # run trajectories until done
        episodes_sum_of_rewards = []
        states = []
        actions = []
        coeffs = []
        for ep in range(self.nb_episodes_per_gradient_step):
            x,_ = env.reset()
            rewards = []
            episode_cum_reward = 0
            while(True):
                a = self.policy.sample_action(torch.as_tensor(x))
                y,r,done,trunc,_ = env.step(a)
                states.append(x)
                actions.append(a)
                rewards.append(r)
                episode_cum_reward += r
                x=y
                if done: 
                    # The condition above should actually be "done or trunc" so that we 
                    # terminate the rollout also if trunc=True.
                    # But then, our return-to-go computation would be biased as we would 
                    # implicitly assume no rewards can be obtained after truncation, which 
                    # is wrong.
                    # We leave it as is for now (which means we will call .step() even 
                    # after trunc=True) and will discuss it later.
                    # Compute returns-to-go
                    discounted_returns = []
                    G_t = 0
                    for t,r in reversed(list(enumerate(rewards))):
                        G_t = r + self.gamma * G_t
                        discounted_returns.append(self.gamma**t * G_t)
                    discounted_returns = list(reversed(discounted_returns))
                    coeffs.extend(discounted_returns)
                    episodes_sum_of_rewards.append(episode_cum_reward)
                    break
        # make loss
        coeffs = torch.tensor(coeffs)
        log_prob = self.policy.log_prob(torch.as_tensor(np.array(states)),torch.as_tensor(np.array(actions)))
        loss = -(coeffs * log_prob).mean()
        # gradient step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return np.mean(episodes_sum_of_rewards)

    def train(self, env, nb_gradient_steps):
        avg_sum_rewards = []
        for ep in trange(nb_gradient_steps):
            avg_sum_rewards.append(self.one_gradient_step(env))
        return avg_sum_rewards

<div class="alert alert-warning">
    
**Exercise:**  
Run your agent for 50 gradient steps on `CartPole-v1`, drawing 10 episodes at each step.  
Experiment with another number of episodes and vary the gradient step size.
</div>

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

#env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = gym.make('CartPole-v1', render_mode="rgb_array")
config = {'gamma': .99,
          'learning_rate': 0.01,
          'nb_episodes_per_gradient_step': 10
         }

pi = policyNetwork(env)
agent = reinforce_agent(config, pi)
returns = agent.train(env,50)
plt.plot(returns)

In [ ]:
import gymnasium as gym
from gymnasium.utils.save_video import save_video

#test_env = gym.make("LunarLander-v3", render_mode="rgb_array_list")
test_env = gym.make("CartPole-v1", render_mode="rgb_array_list")
s,_ = test_env.reset()
with torch.no_grad():
    for t in range(1000):
        a = pi.sample_action(torch.as_tensor(s))
        s2,r,d,trunc,_ = test_env.step(a)
        s = s2
        if d:
            break

save_video(test_env.render(), "videos", fps=test_env.metadata["render_fps"], name_prefix="reinforce_policy")

In [ ]:
from IPython.display import Video
Video("videos/reinforce_policy-episode-0.mp4")

**Comment on the behavior of policy gradient methods (based on the CartPole example).**

One very interesting feature of policy gradient methods, compared to dynamic programming ones, is that they take a somewhat more pragmatic  and easier approach at policy improvement.

Policy gradients make the policy better by reinforcing good actions in visited states, and discouraging actions that led to bad outcomes. In CartPole, this means that exiting the screen on either side will be penalized as much as falling. This happens quite early in the optimization process: as soon as the policy balances the pole, it starts to drift towards the sides of the screen, gets this negative reward, and hence reinforces actions that keep the cart within the screen (the fact that we draw several trajectories for each policy helps a lot for that matter since it enables comparing different actions and reinforcing the ones which belonged to better trajectories).

Converserly, maybe you remember DQN did a great job at quickly learning how to balance the pole, but wasn't so great at avoiding drifting to the edges of the screen. The key difference is that DQN stored all past experience, and that exiting from the side ended up being a rather rare situation within the replay buffer. Also, DQN needed to propagate these bad rewards through the value function (a task greatly enhanced by methods like prioritized experience replay), which is done immediately in REINFORCE because $G_t$ is a Monte Carlo estimator, not a TD(0) one.

So maybe, with ADP methods, we had set up a task that was excessively difficult. The ADP approach is elegant and sample efficient (it retains a long-term memory of samples and solves for the optimal $Q$ value for *all* state and action pairs. But maybe this was more than what was necessary: a local search among policy parameters, without retaining a memory of past experience, but instead local comparison of trajectories and reinforcement of good ones, might be sufficient to achieve efficient learning.

<div class="alert alert-warning">
    
**Exercise:**  
The code above is slightly inefficient as it passes each state through the network twice: one when drawing the action (`sample_action`), the other when computing log probabilities (`log_prob`). This is not critical as the latter is a batched version of this forward pass, but still, we can optimize this a bit. Make this a bit more elegant by writing a `sample_action_and_log_prob` function in the `reinforce_agent` class, which jointly draws the action and computes its log probability, enabling storing the log probabilities along training.
</div>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange

class reinforce_agent:
    def __init__(self, config, policy_network):
        self.device = "cuda" if next(policy_network.parameters()).is_cuda else "cpu"
        self.scalar_dtype = next(policy_network.parameters()).dtype
        self.policy = policy_network
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.99
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = torch.optim.Adam(list(self.policy.parameters()),lr=lr)
        self.nb_episodes_per_gradient_step = config['nb_episodes_per_gradient_step'] if 'nb_episodes_per_gradient_step' in config.keys() else 1

    def sample_action_and_log_prob(self, x):
        probabilities = self.policy(torch.as_tensor(x))
        action_distribution = Categorical(probabilities)
        action = action_distribution.sample()
        log_prob = action_distribution.log_prob(action)
        return action.item(), log_prob
    
    def one_gradient_step(self, env):
        # run trajectories until done
        episodes_sum_of_rewards = []
        log_probs = []
        coeffs = []
        for ep in range(self.nb_episodes_per_gradient_step):
            x,_ = env.reset()
            rewards = []
            episode_cum_reward = 0
            while(True):
                a, log_prob = self.sample_action_and_log_prob(x)
                y,r,d,_,_ = env.step(a)
                log_probs.append(log_prob)
                rewards.append(r)
                episode_cum_reward += r
                x=y
                if d:
                    # compute returns-to-go
                    discounted_returns = []
                    G_t = 0
                    for t,r in reversed(list(enumerate(rewards))):
                        G_t = r + self.gamma * G_t
                        discounted_returns.append(self.gamma**t * G_t)
                    discounted_returns = list(reversed(discounted_returns))
                    coeffs.extend(discounted_returns)
                    episodes_sum_of_rewards.append(episode_cum_reward)
                    break
        # make loss
        coeffs = torch.tensor(coeffs)
        log_probs = torch.cat(log_probs)
        loss = -(coeffs * log_probs).mean()
        # gradient step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return np.mean(episodes_sum_of_rewards)

    def train(self, env, nb_gradient_steps):
        avg_sum_rewards = []
        for ep in trange(nb_gradient_steps):
            avg_sum_rewards.append(self.one_gradient_step(env))
        return avg_sum_rewards

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

#env = gym.make("LunarLander-v3", render_mode="rgb_array")
env = gym.make('CartPole-v1', render_mode="rgb_array")
config = {'gamma': .99,
          'learning_rate': 0.01,
          'nb_episodes_per_gradient_step': 10
         }

pi = policyNetwork(env)
agent = reinforce_agent(config, pi)
returns = agent.train(env,50)
plt.plot(returns)

**The problem of rollout terminations.**

One weakness of REINFORCE is that it requires trajectories that terminate in finite time. This was reasonable enough in the previous example as we let the environment run until `done=True`, but this does not happen in all MDPs. Some environments have no particular termination condition. When we instantiate such environments, we generally set a time limit, but this does not constitute a termination condition, rather a trajectory truncation one. In particular, a Monte Carlo estimate of $\mathbb{E}[G^\pi(s,a)]$ based on a truncated trajectory might be very biased if $s,a$ is encountered at a time step close to the time limit.

With this problem, we reach the limit of Monte Carlo evaluations. In previous chapters, to counter this problem, we have introduced value function estimators and bootstrapping (TD learning of Q-functions being the off-policy, 1-step bootstraping version).

# From Monte Carlo policy gradients to actor-critic algorithms

## Baselines in policy gradients

**Finite sample gradient estimator variance**

The policy gradient features a somehow counter-intuitive property for finite sample estimators.  
Recall the form of the policy gradient estimator:
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_{t=0}^\infty \gamma^t G_t \nabla_\theta \log \pi_\theta(a_t|s_t) \right].$$
Now, suppose we have a single trajectory, 3 time steps long, drawn following $\pi$, with $G_0 = 10$, $\gamma G_1 = 1$, and $\gamma^2 G_2 = 1$.

For the sake of brevity, let's write $\nabla_i = \nabla_\theta \log \pi(a_i|s_i)$. 
So the finite sample Monte Carlo estimator for $\nabla_\theta J(\theta)$ is:
$$10\nabla_0 + 1\nabla_1 + 1\nabla_2.$$

Following this update of $\theta$, we will hopefully see trajectories go through $s_0,a_0$ a lot more often, and through $s_1,a_1$ and $s_2,a_2$ a little bit more often.

Now recall the expected grad-log-prob lemma: we can substract any action-independent *baseline* $b(s)$ from the realization of $G^\pi(s,a)$ and still obtain an unbiased estimate of $\nabla_\theta J(\theta)$, because $\mathbb{E}_{\tau} \left[b(s) \nabla_\theta \log\pi(a|s) \right]=0$:
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \left( \gamma^t G_t - b(s) \right) \nabla_\theta \log\pi(a|s)\right].$$

In particular, we can take a baseline of the form $\gamma^t b(s)$, which enables factoring out $\gamma^t$.
<div class="alert alert-success">

**Monte Carlo policy gradient with baseline**  
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_t \gamma^t \left( G_t - b(s_t) \right) \nabla_\theta \log\pi(a_t|s_t)\right].$$
</div>

<div class="alert alert-warning">
    
**Exercise:**  
Recall the expected grad-log-prob lemma, then use it to prove the equality above.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

The EGLP lemma states that:
$$\mathbb{E}_{a\sim \pi_\theta(s)} [ b \nabla_\theta \log \pi_\theta(a|s) ] = 0$$.
This is true for any quantity $b$ that **does not depend on $a$** and hence can be factored out of the expectation.

So, for instance, taking $b = \gamma^t b(s)$, with $b(s)$ an arbitrary function $\mathcal{S}\rightarrow \mathbb{R}$, yields:
$$\mathbb{E}_{a\sim \pi_\theta(s)} [ \gamma^t b(s) \nabla_\theta \log \pi_\theta(a|s) ] = 0.$$

And so, taking the expected value with respect to $s$, we obtain
$$\mathbb{E}_{s_t\sim p_t, a_t\sim \pi_\theta(s_t)} [ \gamma^t b(s_t) \nabla_\theta \log \pi_\theta(a_t|s_t) ] = 0.$$

And more generally, for any $t$, 
$$\mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} [ \gamma^t b(s_t) \nabla_\theta \log \pi_\theta(a_t|s_t) ] = 0.$$

The Monte Carlo policy gradient was given by:
$$\nabla_\theta J(\theta) = \mathbb{E}_{(s_i,a_i)_{i\in \mathbb{N}}} \left[ \sum_{t=0}^\infty \gamma^t \left(\sum_{t'=t}^\infty \gamma^{t'-t} r(s_{t'},a_{t'}) \right) \nabla_\theta \log \pi_\theta(a_t|s_t) \right] .$$

Substracting the two previous equations yields the result.
</details>

Let's add a constant value of $b(s)=-1$ to all discounted returns. Then our finite sample estimate becomes:
$$9\nabla_0.$$
Let's use $b(s)=-10$.  Then our estimated ascent direction is:
$$-9\nabla_1 -9\nabla_2.$$
Conversely, let's add 20 to all discounted returns. Then the estimate gradient is:
$$30 \nabla_0 + 21\nabla_1 + 21\nabla_2.$$

We have not changed our MDP, nor our policy, but the gradient estimate is drastically different, both in norm and in direction! So what has happened?

What has happened is that we have a **finite** sample of states and actions. The baseline $b(s)$ does not affect the gradient estimate's average in the limit of **infinite** sampling. But depending on the baseline's value, the finite sample estimator will have more or less **variance**.

In the previous example, if we augment the probability of $a$ in a given $s$, then we need to decrease it for another $a'$. If this other $a'$ has not been sampled, then its $\nabla\theta \log\pi_\theta(a'|s)$ cannot participate in the estimated ascent direction which might be off just because it strongly depends on which sample set was drawn.

Ideally, we would like to find a baseline which minimizes variance of the estimator, so that whatever finite sample set we draw, we have a good chance of having a good quality gradient estimate.

**An optimal baseline?**

Let's search for an optimal baseline $b(s)$ in each state, by writing the variance of the gradient estimate. In each state $s$, we discard the $\gamma^t$ factor and write:
$$d(s) = \mathbb{E}_{a\sim \pi} \left[ \left( G^\pi(s,a) - b(s) \right) \nabla_\theta \log\pi(a|s)\right].$$
The estimated ascent direction is $\mathbb{E}_{\tau} [\gamma^t d(s_t)]$.

To simplify notations, we will drop all mentions of $s$ ($s$ is fixed), and will write $\nabla(a)$ for $\nabla_\theta \log\pi(a|s)$. Let us define a proxy for the variance of vector $d$:
$$Var[d] = \mathbb{E}_{a\sim \pi} \left[ \left( G^\pi(a)-b \right)^2 \nabla(a)^2 \right] - \left[ \mathbb{E}_{a\sim \pi} \left[ \left( G^\pi(a)-b \right) \nabla(a) \right] \right]^2.$$
(this is related to the trace of the covariance matrix but we won't go into details on that for now)

We wish to find $b$ that minimizes $Var[d]$. A first remark is that the second term is actually equal to $\left[ \mathbb{E}_{a\sim \pi} \left[ \left( G^\pi(a) \right) \nabla(a) \right] \right]^2$ and does not depend on $b$. We get that:
\begin{align*}
\frac{\mathrm{d} Var[d]}{\mathrm{d}b} &= \frac{\mathrm{d}}{\mathrm{d}b} \mathbb{E}_{a\sim \pi} \left[ \left( G^\pi(a)-b \right)^2 \nabla(a)^2 \right],\\
 &= \frac{\mathrm{d}}{\mathrm{d}b} \Big[\mathbb{E}_{a\sim \pi} \left[G^\pi(a)^2 \nabla(a)^2\right] -2 b\mathbb{E}_{a\sim \pi} \left[G^\pi(a) \nabla(a)\right] + b^2 \mathbb{E}_{a\sim \pi} \left[\nabla(a)^2\right] \Big],\\
 &= -2 \mathbb{E}_{a\sim \pi} \left[G^\pi(a) \nabla(a)^2\right] + 2 \mathbb{E}_{a\sim \pi} \left[b\nabla(a)^2\right].
\end{align*}

And so the value that cancels this gradient is:
$$b = \frac{\mathbb{E}_{a\sim \pi} \left[G^\pi(a) \nabla(a)^2\right]}{\mathbb{E}_{a\sim \pi} \left[\nabla(a)^2\right]}.$$

Suppose $\nabla(a)^2$ is evenly distributed for all $a$. Then this fraction boils down to $b = \mathbb{E}_{a\sim \pi} \left[G^\pi(a)\right]$, which means $b(s) = v^\pi(s)$. 

But there is no  particular reason for $\nabla(a)^2$ to be evenly distributed for all $a$. This would mean all $\log \pi_\theta(a|s)$ have the same amplitude of rate of change with respect to $\theta$, across all $a$, which is quite unlikely.  
So the optimal baseline $b(s)$ is a weighted average across actions of $G^\pi(s,a)$ but with actions sampled according to a distribution which depends both on $\pi(a|s)$ and $\nabla_\theta \log\pi(a|s)$. In other words: since $q^\pi(s,a)$ is the expectation of $G^\pi(s,a)$, the optimal baseline is almost $v^\pi(s)$ but instead of weighting every action with $\pi(a|s)$, it is additionally weighted by the norm of the gradient of the policy's log probabilities.

**The value function as a baseline in REINFORCE**

Although it is not optimal, a close enough choice for a baseline is the policy's value function $v^\pi$.

REINFORCE with a value function baseline is quite straightforward. It introduces a value function approximator $v_w$, which tries to fit $v^\pi$.
1. Initialize policy parameter $\theta$ and value function parameter $w$
2. Generate a trajectory by playing $\pi$: $s_0,a_0,r_0,...s_{T}$
3. For $t\in [1, 2, … , T]$:
    1. Estimate return $G_t$
    2. Update value parameter: $w \leftarrow w + \alpha (G_t-v_w(s_t)) \nabla_w v_w(s_t)$
    3. Update policy parameter: $\theta \leftarrow \theta + \alpha \gamma^t (G_t-v_w(s_t)) \nabla_\theta \log \pi(a_t|s_t)$

Note that the algorithm above uses a Monte Carlo method to learn $v^\pi$ (line B). It lets the episode unroll until termination, before summing all rewards to obtain the realizations of $G_t$ for each time step $t$.

## Advantage actor-critic

As a next step, it seems natural to replace the Monte Carlo learning of $v^\pi$ by a TD method. As we have seen in the chapter on value function learning, we can expect this to have the strong benefit of strongly reducing the variance of the value function estimates (at the cost of a possible bias in the estimator). In turn, this variance reduction is expected to provide better policy gradients.

It is very tempting to replace $G_t$ (the random variable for the sum of returns) by $q^\pi(s_t,a_t)$ (the function that estimates this random variable's expectation). The previous derivation of the Monte Carlo policy gradient, where we explicitly wrote $\sum_{t'\geq t} \gamma^{t'-t} r_t$, makes this legitimate. Then, since REINFORCE with a value function baseline uses $G_t-v^\pi(s_t)$ weights, it implictly introduces an *advantage* estimation problem, where the advantage is the function defined as:
$$q^\pi(s,a) - v^\pi(s).$$

Note: we could introduce a function $a^\pi(s,a) = q^\pi(s,a) - v^\pi(s)$ as many authors do. We refrain from this, just to avoid any confusion between the notation of the advantage $a^\pi$ and the actions $a$.

Remember that $q^\pi(s,a) = \mathbb{E}_{s'\sim p(s,a)} [r(s,a,s') + \gamma v^\pi(s')]$. So, given a sample $(s,a,r,s')$, a sample estimate of the advantage function is $r + \gamma v^\pi(s') - v^\pi(s)$. So learning $v^\pi$ provides a way to obtain sample estimates of the advantage function. 

The change from $G_t - v^\pi(s_t)$ to $r_t + \gamma v^\pi(s_{t+1}) - v^\pi(s_t)$ is an important one, that needs to be emphasized. $G_t$ was a **Monte Carlo estimate** of the payoff, while $r_t + \gamma v^\pi(s_{t+1})$ is a **bootstrap estimate** of the same quantity. By introducing the bootstrap term $v^\pi(s_{t+1})$, one moves away from **Monte Carlo policy gradients**, and into **actor-critic methods**.

This yields the advantage policy gradient:
<div class="alert alert-success">

**Advantage actor-critic policy gradient**  
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_t \gamma^t \left[r(s_t,a_t,s_{t+1}) + \gamma v^\pi(s_{t+1}) - v^\pi(s_t)\right] \nabla_\theta \log\pi(a_t|s_t)\right]$$
</div>

Interestingly, the samples of the $G_t-v^\pi(s_t)$ weights in the advantage actor-critic policy gradients are **temporal differences** $\delta_t = r_t + \gamma v^\pi(s_{t+1}) - v^\pi(s_t)$. These are used in the policy gradient definition, but also participate in the learning of $v_w$. Overall, **actor-critic methods** are methods that are based on **bootstrap estimates** of the payoff, enabling **online learning** of the value function and **online improvement** of the policy (i.e. learning that does not require episode termination).

<div class="alert alert-success">

**Actor-critic methods**  
Actor-critic methods are the family of algorithms that store both a value function (either $v$ or $q$) and a policy $\pi$, and use bootstrap estimates to enable online learning of $v^\pi$ or $q^\pi$ by temporal difference learning, and online improvement of $\pi$ by using the policy gradient.
</div>

Discussion:  
Advantage actor-critic is not the only actor-critic algorithm out there. As stated above, we talk about actor-critic algorithms as soon as the weights use a boostrap estimate instead of a Monte Carlo one, therefore enabling online learning. Thus, one could simply replace $G_t$ by $q^\pi(s,a)$ in vanilla REINFORCE (without baselines). This would yield a gradient estimate of the form 
$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau} \left[ \sum_t \gamma^t q^\pi(s_t,a_t) \nabla_\theta \log\pi(a_t|s_t)\right].$$
This gradient estimate could be called the "Q actor-critic policy gradient", and $q^\pi$ could be estimated using some TD(0) method on the collected samples.  
This gradient has little interest however, as it would suffer from the high variance problem, and the advantage actor-critic policy gradient will likely be a better choice.  
The point of this short remark was simply to stress out that actor-critic methods can use different replacements for $G_t$, and that the advantage-based one is the one that uses the value function as a baseline.  
Another important comment is that TD methods can be either on-policy or off-policy. For instance, we saw in previous chapters that TD($\lambda$) for $v^\pi$ was on-policy, while TD(0) for $q^\pi$ was off-policy. But in all cases, the policy improvement step, using the policy gradient, is on-policy: it requires samples drawn exclusively by rolling out trajectories playing $\pi$.

**The advantage actor-critic algorithm**

We noted that the temporal difference at each time step $\delta = r + \gamma v^\pi(s') - v^\pi(s)$ is an estimate of the advantage $q^\pi(s,a) - v^\pi(s)$. Using this remark, a simple one-step advantage actor-critic method based on TD(0) and a value function $v_w$ goes as follows:  
<div class="alert alert-success">

**Advantage actor-critic**  
For each step $t$ in an episode:
1. In $s$, draw $a \sim \pi$
2. Observe $r, s'$
3. Compute $\delta = r + \gamma v_w(s') - v_w(s)$
4. Update critic's parameters (TD(0) step) $w \leftarrow w + \alpha \delta \nabla_w v_w(s)$
5. Update actor's parameters (advantage policy gradient) $\theta \leftarrow \theta + \alpha \gamma^t \delta \nabla_\theta \log \pi(a|s)$
6. $s\leftarrow s'$ and repeat
</div>

This directly translates into a batch algorithm:
<div class="alert alert-success">

**Advantage actor-critic, batch version**  
1. Sample trajectories $\{s_t,a_t,r_t,s'_t\}$ from $\pi$
2. Fit $v_w$ to the (bootstrapped) returns $r_t + \gamma v_w(s'_t)$
3. Get advantage estimate $\delta_t = r_t + \gamma v_w(s'_t) - v_w(s_t)$
4. Update policy using ascent direction $\sum_t \gamma^t \delta_t \nabla_\theta \log \pi(a_t|s_t)$
</div>

An interesting feature of this algorithm is that it does not require to store experience samples from past policies. Since fitting of $v_w$ is an on-policy problem, it operates on the last collected batch of samples and does not retain the experience replay buffers we encountered with ADP methods.

Note that step 2 of the batch version can be repeated several times before we move on to step 3, in order to apply $\mathbb{T}^\pi$ several times and obtain better estimates of $v^\pi$. This is rarely done in practice but could prove useful under the right circumstances.

Overfitting of $v_w$ to the samples it is trained upon is not an issue since $v_w$ will only ever be evaluated on these same samples.

To reduce the bias in learning $v_w$, instead of one-step targets $r_t + \gamma v_w(s'_t)$, one can store and use n-step returns $\sum_{t'=t}^{t+n-1} \gamma^{t'-t} r_{t'} + \gamma^{n} v_w(s'_{t+n-1})$ for each $s_i$.

## A2C: Deep Advantage Actor-Critic

In 2016, Mnih et al. introduced a set of algorithms using asynchronous environments in the **[Asynchronous Methods for Deep Reinforcement Learning](https://arxiv.org/abs/1602.01783)** paper. Among these methods, one is called A3C for Asynchronous Advantage Actor-Critic. 
In a nutshell, A3C is an advantage actor-critic method where multiple policy workers in parallel environments independently update a global value function. One supposed key benefit of having asynchronous actors was to promote exploration of the state space. It later turned out, the full asynchronicity of independent policies did not bring a significant performance advantage while inducing some computational overhead. A2C is A3C's little sibling, using a single (hence synchronous) policy.

At each policy update step, A2C runs a number of rollouts with $\pi_\theta$, collecting samples.

It uses bootstrapped n-step Monte Carlo returns to estimate the advantage in each state-action pair:
$$\delta(s_t,a_t;w) = \sum_{t'=t}^{T-1} \gamma^{t'-t} r_{t'} + \gamma^{T-t} v_w(s_{T}) - v_w(s_t),$$
where $T$ is the termination time of a rollout (note the $v_w(s_{T})$ bootstrap value at the end of the rollout).

The value function learning takes a gradient step to minimize:
$$L_{critic}(w) = \sum_t \delta(s_t,a_t;w)^2.$$

The policy learning step takes a gradient step to maximize:
$$L_{actor}(\theta)=\sum_t \left[ \gamma^t \delta(s_t,a_t;w) \nabla_\theta \log \pi_\theta(a_t|s_t) + \beta \nabla_\theta \mathcal{H}(\pi_\theta(s_t) \right].$$
The first term in this sum is the usual policy gradient estimate. The second one is the entropy of the policy. The intention is to promote exploration by retaining randomness in the policy.

So, in short, A2C's pseudo code is:
1. Collect trajectories $\{s_t,a_t,r_t,s'_t\}$ from $\pi$
2. Define $\delta(s_t,a_t;w) = \sum_{t'=t}^{T-1} \gamma^{t'-t} r_{t'} + \gamma^{T-t} v_w(s_{T}) - v_w(s_t)$
3. Take a gradient descent step on $L_{critic}(w)$ and a gradient ascent step on $L_{actor}(\theta)$.

In the implementation of REINFORCE, the coefficients $\gamma^t G_t$ were obtained by summing rewards backwards in time from the terminal state. Since A2C uses the value function as a baseline, these coefficients are now advantages and their recursive computation needs a bit of thinking.

<div class="alert alert-warning">
    
**Exercise:**  
Let $\delta_t = \sum_{t'=t}^{T-1} \gamma^{t'-t} r_{t'} + \gamma^{T-t} v_w(s_{T}) - v_w(s_t)$ denote the advantage sample for state-action pair $(s_t,a_t)$. What is the expression of $\delta_{t-1}$ given $\delta_t$?  
Derive a recursive computation method to obtain all $\delta_t$.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

\begin{align*}
\delta_{t-1} &= \sum_{t'=t-1}^{T-1} \gamma^{t'-(t-1)} r_{t'} + \gamma^{T-t} v_w(s_{T}) - v_w(s_{t-1})\\
 &= r_{t-1} + \gamma \left[ \sum_{t'=t}^{T-1} \gamma^{t'-t} r_{t'} + \gamma^{T-t} v_w(s_{T}) \right] - v_w(s_{t-1})\\
 &= r_{t-1} + \gamma \left[ \delta_t + v_w(s_t) \right] - v_w(s_{t-1})\\
 &= r_{t-1} + \gamma v_w(s_t) - v_w(s_{t-1}) + \gamma \delta_t
\end{align*}

So we can start with $\delta_T = 0$ and recursively compute $\delta_{t-1} = r_{t-1} + \gamma v_w(s_t) - v_w(s_{t-1}) + \gamma \delta_t$ backwards in time, until we reach $t=0$. This will yield the sequence of $\delta_t$ defined in A2C's pseudo code.
</details>

<div class="alert alert-warning">
    
**Exercise:**  
Declare a critic network.
</div>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class valueNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        state_dim = env.observation_space.shape[0]
        n_action = env.action_space.n
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(dim=0)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

<div class="alert alert-warning">
    
**Exercise:**  
Implement an A2C algorithm.
</div>

In [ ]:
# DEPRECATED - REMOVE WHEN NEW VERSION WORKS

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange

class a2c_agent:
    def __init__(self, config, policy_network, value_network):
        self.device = "cuda" if next(policy_network.parameters()).is_cuda else "cpu"
        self.scalar_dtype = next(policy_network.parameters()).dtype
        self.policy = policy_network
        self.value = value_network
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.99
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = torch.optim.Adam(list(self.policy.parameters()) + list(self.value.parameters()),lr=lr)
        self.nb_episodes_per_gradient_step = config['nb_episodes_per_gradient_step'] if 'nb_episodes_per_gradient_step' in config.keys() else 1
        self.entropy_coefficient = config['entropy_coefficient'] if 'entropy_coefficient' in config.keys() else 0.001

    def sample_action(self, x):
        probabilities = self.policy(torch.as_tensor(x))
        action_distribution = Categorical(probabilities)
        action = action_distribution.sample()
        log_prob = action_distribution.log_prob(action)
        entropy = action_distribution.entropy()
        return action.item(), log_prob, entropy
    
    def one_gradient_step(self, env):
        # run trajectories until done
        episodes_sum_of_rewards = []
        log_probs = []
        coeffs = []
        values = []
        entropies = []
        for ep in range(self.nb_episodes_per_gradient_step):
            x,_ = env.reset()
            rewards = []
            episode_cum_reward = 0
            while(True):
                a, log_prob, entropy = self.sample_action(x)
                y,r,d,trunc,_ = env.step(a)
                values.append(self.value(torch.as_tensor(x)).squeeze(dim=0))
                log_probs.append(log_prob)
                entropies.append(entropy)
                rewards.append(r)
                episode_cum_reward += r
                x=y
                if d or trunc:
                    # compute returns-to-go
                    discounted_returns = []
                    G_t = torch.tensor([0])
                    if trunc:
                        G_t = self.value(torch.as_tensor(x)).squeeze(dim=0).detach() # detach ?
                    for t,r in reversed(list(enumerate(rewards))):
                        G_t = r + self.gamma * G_t
                        discounted_returns.append(self.gamma**t * G_t)
                    discounted_returns = list(reversed(discounted_returns))
                    coeffs.extend(discounted_returns)
                    episodes_sum_of_rewards.append(episode_cum_reward)
                    break
        # make loss        
        coeffs = torch.cat(coeffs)
        values = torch.cat(values)
        log_probs = torch.cat(log_probs)
        entropies = torch.cat(entropies)
        advantages = coeffs - values
        pg_loss = -(advantages.detach() * log_probs).mean()
        entropy_loss = -entropies.mean()
        critic_loss = advantages.pow(2).mean()
        loss = pg_loss + critic_loss + self.entropy_coefficient * entropy_loss
        # gradient step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return np.mean(episodes_sum_of_rewards)

    def train(self, env, nb_gradient_steps):
        avg_sum_rewards = []
        for ep in trange(nb_gradient_steps):
            avg_sum_rewards.append(self.one_gradient_step(env))
        return avg_sum_rewards

In [ ]:
# NEW VERSION - WIP

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import trange

class a2c_agent:
    def __init__(self, config, policy_network, value_network):
        self.device = "cuda" if next(policy_network.parameters()).is_cuda else "cpu"
        self.scalar_dtype = next(policy_network.parameters()).dtype
        self.policy = policy_network
        self.value = value_network
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.99
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = torch.optim.Adam(list(self.policy.parameters()) + list(self.value.parameters()),lr=lr)
        self.nb_episodes_per_gradient_step = config['nb_episodes_per_gradient_step'] if 'nb_episodes_per_gradient_step' in config.keys() else 1
        self.entropy_coefficient = config['entropy_coefficient'] if 'entropy_coefficient' in config.keys() else 0.001

    def sample_action(self, x):
        probabilities = self.policy(torch.as_tensor(x))
        action_distribution = Categorical(probabilities)
        action = action_distribution.sample()
        log_prob = action_distribution.log_prob(action)
        entropy = action_distribution.entropy()
        return action.item(), log_prob, entropy
    
    def one_gradient_step(self, env):
        # run trajectories until done
        episodes_sum_of_rewards = []
        log_probs = []
        coeffs = []
        values = []
        entropies = []
        for ep in range(self.nb_episodes_per_gradient_step):
            x,_ = env.reset()
            rewards = []
            episode_cum_reward = 0
            while(True):
                a, log_prob, entropy = self.sample_action(x)
                y,r,d,trunc,_ = env.step(a)
                values.append(self.value(torch.as_tensor(x)).squeeze(dim=0))
                log_probs.append(log_prob)
                entropies.append(entropy)
                rewards.append(r)
                episode_cum_reward += r
                x=y
                if d or trunc:
                    # compute returns-to-go
                    discounted_returns = [] # remove
                    advantages = []
                    final_value = (1.-done)*self.value(torch.as_tensor(x)).squeeze(dim=0)
                    for t,r in reversed(list(enumerate(rewards))): # it becomes useful to loop on t now
                        avantage = # TODO !
                        discounted_returns.append(self.gamma**t * G_t) # remove
                    discounted_returns = list(reversed(discounted_returns))
                    coeffs.extend(discounted_returns)
                    episodes_sum_of_rewards.append(episode_cum_reward)
                    break
        # make loss        
        coeffs = torch.cat(coeffs)
        values = torch.cat(values)
        log_probs = torch.cat(log_probs)
        entropies = torch.cat(entropies)
        advantages = coeffs - values
        pg_loss = -(advantages.detach() * log_probs).mean()
        entropy_loss = -entropies.mean()
        critic_loss = advantages.pow(2).mean()
        loss = pg_loss + critic_loss + self.entropy_coefficient * entropy_loss
        # gradient step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return np.mean(episodes_sum_of_rewards)

    def train(self, env, nb_gradient_steps):
        avg_sum_rewards = []
        for ep in trange(nb_gradient_steps):
            avg_sum_rewards.append(self.one_gradient_step(env))
        return avg_sum_rewards

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

env = gym.make("LunarLander-v3", render_mode="rgb_array")
#env = gym.make('CartPole-v1', render_mode="rgb_array")
config = {'gamma': .99,
          'learning_rate': 0.01,
          'nb_episodes_per_gradient_step': 10,
          'entropy_coefficient': 1e-3
         }

pi = policyNetwork(env)
V  = valueNetwork(env)
agent = a2c_agent(config, pi, V)
returns = agent.train(env,150)
plt.plot(returns)

In [ ]:
import gymnasium as gym
from gymnasium.utils.save_video import save_video

test_env = gym.make("LunarLander-v3", render_mode="rgb_array_list")
#test_env = gym.make("CartPole-v1", render_mode="rgb_array_list")
s,_ = test_env.reset()
with torch.no_grad():
    for t in range(1000):
        a = pi.sample_action(torch.as_tensor(s))
        s2,r,d,trunc,_ = test_env.step(a)
        s = s2
        if d:
            break

save_video(test_env.render(), "videos", fps=test_env.metadata["render_fps"], name_prefix="a2c_policy")

In [ ]:
from IPython.display import Video
Video("videos/a2c_policy-episode-0.mp4")

# Summary

In this chapter, we have considered the problem of direct policy search. We have:
- cast this problem as a gradient ascent one,
- introduced Monte Carlo policy gradients,
- derived their direct application as the REINFORCE algorithm,
- tackled the problem of gradient variance through the introduction of baselines.
- Since baselines required estimating a value function, we used TD-learning to learn them time-steps after time step,
- we lead to actor-critic algorithms,
- and in particular we have introduced the recent A2C algorithm.

Of course there are still lots of things to cover in the literature on Monte Carlo policy gradients and more generally policy search. The homework will guide you through some of these important topics, which should provide you with an up-to-date view on the current state-of-the-art in direct policy search.

# Homework

The exercises below are here to help you play with the concepts introduced above, to better grasp them. They also introduce additional important notions. They are not optional to reach the class goals. Often, the provided answer reaches out further than the plain question asked and provides comments, additional insights, or external references.

## Running rollouts in parallel

One key feature of policy gradient algorithms is that they are on-policy: they require the data to have been collected by the current policy, and discard this data once a gradient step is taken. Although this might seem sample inefficient, it can turn out to be an acceptable compromise if the policy gradient steps take the policy towards good returns quickly enough.

A direct consequence of this on-policy property, is that policy gradients don't use experience replay buffers. Instead, they can take advantage of parallel computation to collect samples. We saw that running several rollouts with the same policy greatly helped in reducing the gradient's variance.

<div class="alert alert-warning">
    
**Exercise:**  
Gymnasium provides a [vectorized environments](https://gymnasium.farama.org/api/vector/) class, which enables running multiple independent copies of the same environment in parallel, for a certain number of steps. Modify the REINFORCE and A2C classes to use a vector of environments.
</div>

In [ ]:
# DEMO CODE OF VECTORIZED ENVIRONMENTS
import gymnasium as gym
import numpy as np

## Make vector env and simulate one step
#envs = gym.vector.make("CartPole-v1", num_envs=10)
envs = gym.make_vec("CartPole-v1", num_envs=10)
## Reset all environments
states, _ = envs.reset(seed=42)
#print(states)
## Choose actions
actions = envs.action_space.sample()
#print(actions)
## Step
next_states, rewards, termination, truncation, infos = envs.step(actions)
#print(observations)
#print(infos)

## Warning, when an environment terminates or is truncated, it automatically resets 
## so that you can keep calling .step() and it does not wait idly until others have 
## finished.
## When this happens, the next state returned by .step() is the result of the reset
## and the last state of the previous trajectory is accessed through the 
## infos['final_observation'] variable.
## This also means you cannot use the termination or truncation flags anymore to run 
## post-trajectories computations.

states = next_states
for t in range(50):
    actions = envs.action_space.sample()
    next_states, rewards, termination, truncation, infos = envs.step(actions)
    if any(termination):
        index = np.where(termination == True)
        print("Termination vector:", termination, index)
        index = index[0][0]
        print("Env", index, "has been reset! Here is the last transition:")
        print("starting state =", states[index])
        print("ending state   =", infos['final_observation'][index])
        print("reset state    =", next_states[index])
        break
    states = next_states

In [ ]:
#%load solutions/no_solution_yet.py

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

#env = gym.make("LunarLander-v2", render_mode="rgb_array")
env = gym.make('CartPole-v1', render_mode="rgb_array")
config = {'gamma': .99,
          'learning_rate': 0.01,
          'nb_episodes': 10,
          'nb_sim_steps_per_update': 500
         }

pi = policyNetwork(env)
agent = reinforce_agent(config, pi)
returns = agent.train(env,50)
plt.plot(returns)

## Policy gradients on continuous action domains

REINFORCE and A2C with Gaussian policies

## Policy gradients for the finite horizon criterion

Derive the PG for the total reward, finite horizon criterion.

Implement a REINFORCE algorithm.

## Generalized advantage estimation

[Generalized Advantage Estimation (Schulman et al., 2016)](https://arxiv.org/abs/1506.02438)

## From off-policy policy gradients to TRPO

[Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477)

Policy gradient methods are intrinsically on-policy algorithms, as states and actions need to be sampled according to $\rho^\pi$ and $\pi$ respectively. But could we make an off-policy version of this gradient estimator?

More precisely, could we use data collected by some policy $\pi$, and the corresponding advantage function $A^{\pi}$, to still compute the gradient of $J(\theta')$?

In this section, we try to write an optimization problem whose gradient can be used to optimize $J(\theta')$ while using samples from $\pi_\theta$.

**$J(\theta')$ as an expression of $\pi$ and $A^\pi$**

Recall that:
$$J(\theta) = \mathbb{E}_{s_0 \sim p_0} \left[ V^\pi(s_0) \right].$$

One can remark that this is also:
$$J(\theta) = \mathbb{E}_{\substack{s_0 \sim p_0\\ s_{t>0}\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ V^\pi(s_0) \right].$$

Since the starting state distribution is assumed fixed, we can write:
$$J(\theta) = \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ V^\pi(s_0) \right].$$

It is tempting to compare this expression of $J(\theta)$ which artificially uses $\pi'$, with that of $J(\theta')$ and make the advantage function of $\pi$ appear, with the intention to write $J(\theta')$ as an expression of $J(\theta)$ and this advantage function. 

Let's first remark that:
\begin{align*}
J(\theta) &= \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ \sum_{t=0}^\infty \gamma^t V^\pi(s_t) - \sum_{t=1}^\infty \gamma^t V^\pi(s_t) \right],\\
 &= \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ \sum_{t=0}^\infty \gamma^t \left( V^\pi(s_t) - \gamma V^\pi(s_{t+1}) \right) \right].
\end{align*}

Then recall that $J(\theta') = \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ \gamma^t r(s_t,a_t) \right]$. So:
\begin{align*}
J(\theta') - J(\theta) &= \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ \sum_{t=0}^\infty \gamma^t \left( r(s_t, a_t) + \gamma V^\pi(s_{t+1}) - V^\pi(s_t) \right) \right],\\
 &= \mathbb{E}_{\substack{s_t\sim p_t(\cdot|\pi')\\ a_t\sim \pi'}} \left[ \sum_{t=0}^\infty \gamma^t A^\pi(s_t,a_t) \right].
\end{align*}

In other terms:
$$J(\theta') = J(\theta) + \mathbb{E}_{s_t\sim p_t(\cdot|\pi')} \left[ \mathbb{E}_{a_t\sim \pi'} \left[ \sum_{t=0}^\infty \gamma^t A^\pi(s_t,a_t) \right]\right].$$

We have $J(\theta')$ as an expression of $A^\pi$, but we still require sampling according to $\pi'$. Let us first replace all the $p_t$ with the state occupancy measure:
\begin{align*}
J(\theta') &= J(\theta) + \mathbb{E}_{s_t\sim p_t(\cdot|\pi')} \left[ \mathbb{E}_{a_t\sim \pi'} \left[ \sum_{t=0}^\infty \gamma^t A^\pi(s_t,a_t) \right]\right]\\
 &= J(\theta) + \sum_{t=0}^\infty \gamma^t \mathbb{E}_{s_t\sim p_t(\cdot|\pi')} \left[ \mathbb{E}_{a_t\sim \pi'} \left[ A^\pi(s_t,a_t) \right]\right] \\
 &= J(\theta) + \mathbb{E}_{s\sim \rho^{\pi'}} \left[ \mathbb{E}_{a\sim \pi'} \left[ A^\pi(s,a) \right]\right].
\end{align*}

Now we would like to sample according to $\rho^{\pi}$ and $\pi$ instead of $\rho^{\pi'}$ and $\pi'$ so we can use importance sampling:
$$J(\theta') = J(\theta) + \mathbb{E}_{s\sim \rho^{\pi}} \left[ \mathbb{E}_{a\sim \pi} \left[ \frac{\rho^{\pi'}(s)}{\rho^{\pi}(s)} \frac{\pi'(a|s)}{\pi(a|s)} A^\pi(s,a) \right]\right].$$

This is true as long as the distributions due to $\pi$ have non-zero mass where the distributions due to $\pi'$ have non-zero mass.

So in theory, we could calculate our policy score $J(\theta')$ if we knew the ratio $\frac{\rho^{\pi'}(s)}{\rho^{\pi}(s)}$. But it is very unclear how to actually compute this ratio.

One interesting remark is that neural network updates make very small gradient steps on the policy parameters. So it is likely that between two consecutive gradient steps, the policy does not change much, and hopefully the state distribution does no change much either. In that case, we could neglect this ratio and write $\frac{\rho^{\pi'}(s)}{\rho^{\pi}(s)} \approx 1$, which would enable keeping samples collected by $\pi$ in memory and re-using them in:
$$J(\theta') \approx \mathbb{E}_{s\sim\rho^{\pi}} \mathbb{E}_{a\sim \pi} \left[ \frac{\pi'(a|s)}{\pi(a|s)}A^\pi(s,a)\right].$$

**When is $\frac{\rho^{\pi'}(s)}{\rho^{\pi}(s)} \approx 1$?**

**The constrained optimization problem and its Lagrangian**

**Natural policy gradients**

**TRPO**

## PPO

[Proximal Policy Optimization Algorithms](https://arxiv.org/abs/1707.06347)

## Gradient-free policy search

[Evolution Strategies as a Scalable Alternative to Reinforcement Learning](https://arxiv.org/abs/1703.03864)  
[Back to Basics: Benchmarking Canonical Evolution Strategies for Playing Atari](https://arxiv.org/abs/1802.08842)